# Introduction

Suggested readings are [Chapter 1: What is Deep Learning?](https://colab.research.google.com/github/fastai/fastbook/blob/master/01_intro.ipynb) and PyTorch Chapters 1-4.


Important note: fast ai is running most things on the GPU, so needs to be run in Google Colab. Remember to switch the runtime type at the start of each session!

## Fast.ai Chapter 1

This section is a basic introduction and background for neural networks and the types of problems that fast.ai (or PyTorch in general) can solve. The examples and syntax will be helpful to revist when working on problems.

## PyTorch Chapters 1 & 2

The table of contents provides a decent overview of the Chapter 1

1. Introducing deep learning and the PyTorch Library 
    1.  The deep learning revolution 
    2.  PyTorch for deep learning 
    3.  Why PyTorch? The deep learning competitive landscape 
    4.  An overview of how PyTorch supports deep learning projects
    5.  Hardware and software requirements Using Jupyter Notebooks

Chapter 2 introduces `torch.Tensor` and several pretrained models. These models can be downloaded an run without CUDA.

In [47]:
from torchvision import models

dir(models)
alexnet = models.AlexNet()
resnet = models.resnet101(pretrained=True)

In [48]:
from torchvision import transforms
preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )])

In [62]:
# testing on my own images
# import images
from PIL import Image
holly=Image.open("images/holly.jpeg")
blanche=Image.open("images/blanche.jpeg")
maggie=Image.open("images/maggie.jpeg")
keela=Image.open("images/keela.jpeg")

In [63]:
#preprocess images

holly_t = preprocess(holly)
blanche_t = preprocess(blanche)
maggie_t = preprocess(maggie)
keela_t = preprocess(keela)

In [64]:
#reshape, crop, and normalize the input tensor in a way that the network expects.

import torch

batch_holly_t = torch.unsqueeze(holly_t,0)
batch_blanche_t = torch.unsqueeze(blanche_t, 0)
batch_maggie_t = torch.unsqueeze(maggie_t, 0)
batch_keela_t = torch.unsqueeze(keela_t,0)

In [52]:
# put network in eval mode

resnet.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [53]:
# load labels

with open('images/imagenet_classes.txt') as f:
    labels = [line.strip() for line in f.readlines()]

In [54]:
# interferrence for holly photo
out_holly = resnet(batch_holly_t)

# find the index corresponding to maximum score from output
_, index = torch.max(out_holly, 1)

percentage = torch.nn.functional.softmax(out_holly, dim=1)[0] * 100
labels[index[0]], percentage[index[0]].item()

('Blenheim spaniel', 99.6023178100586)

A Google search for `Blenheim spaniel` gives `Blenheim King Charles spaniel`, which is pretty close! Blenheim is the color pattern, and there's not much difference between a King Charles and Cavalier King Charles Spaniel.

In [55]:
# use sort to find the other options

_, indices = torch.sort(out_holly, descending=True)
[(labels[idx], percentage[idx].item()) for idx in indices[0][:5]]

[('Blenheim spaniel', 99.6023178100586),
 ('cocker spaniel, English cocker spaniel, cocker', 0.20693804323673248),
 ('Sussex spaniel', 0.1252194344997406),
 ('Japanese spaniel', 0.018629075959324837),
 ('Welsh springer spaniel', 0.018065497279167175)]

In [57]:
# interferrence for blanche photo
out_blanche = resnet(batch_blanche_t)

# find the index corresponding to maximum score from output
_, index = torch.max(out_blanche, 1)

percentage = torch.nn.functional.softmax(out_blanche, dim=1)[0] * 100
labels[index[0]], percentage[index[0]].item()

('Labrador retriever', 81.59757995605469)

Also pretty good! Especially since it is likely only trained on pure breds.

In [58]:
# use sort to find the other options

_, indices = torch.sort(out_blanche, descending=True)
[(labels[idx], percentage[idx].item()) for idx in indices[0][:5]]

[('Labrador retriever', 81.59757995605469),
 ('golden retriever', 11.420183181762695),
 ('redbone', 2.1440072059631348),
 ('kelpie', 0.8565823435783386),
 ('Chesapeake Bay retriever', 0.399454265832901)]

Even better! That's probably roughly the percentages a human would guess.

In [59]:
# interferrence for maggie photo
out_maggie = resnet(batch_maggie_t)

# find the index corresponding to maximum score from output
_, index = torch.max(out_maggie, 1)

percentage = torch.nn.functional.softmax(out_maggie, dim=1)[0] * 100
labels[index[0]], percentage[index[0]].item()

('Doberman, Doberman pinscher', 19.89142608642578)

Ok, that one is terrible! 

In [60]:
# use sort to find the other options

_, indices = torch.sort(out_maggie, descending=True)
[(labels[idx], percentage[idx].item()) for idx in indices[0][:5]]

[('Doberman, Doberman pinscher', 19.89142608642578),
 ('Great Dane', 10.994453430175781),
 ('Mexican hairless', 5.362974643707275),
 ('black-and-tan coonhound', 4.684861660003662),
 ('kelpie', 4.482672214508057)]

There are a lot of hounds in the database, but it is interesting that the model seems to be trying various black-and-tan options for a spotted dog. Also, the model is guessing a wide range of sizes!

In [65]:
# interferrence for keela photo
out_keela = resnet(batch_keela_t)

# find the index corresponding to maximum score from output
_, index = torch.max(out_keela, 1)

percentage = torch.nn.functional.softmax(out_keela, dim=1)[0] * 100
labels[index[0]], percentage[index[0]].item()

('Irish water spaniel', 48.229610443115234)

That's definitely the closest of the options in the dataset!

In [66]:
# use sort to find the other options

_, indices = torch.sort(out_keela, descending=True)
[(labels[idx], percentage[idx].item()) for idx in indices[0][:5]]

[('Irish water spaniel', 48.229610443115234),
 ('Sussex spaniel', 14.122137069702148),
 ('miniature poodle', 9.125435829162598),
 ('cocker spaniel, English cocker spaniel, cocker', 6.577445030212402),
 ('standard poodle', 6.206376552581787)]